In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "DavidGI23200/recepie_llm_fine_tuned_with_lora"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model = model.to(device)

In [4]:
def predict_recipe(title_ingredients, tokenizer, device, model):
    """
    Generate recipe directions from title and ingredients.
    Format expected by the model: 'Title: <title> Ingredients: <comma-separated ingredients>'
    """
    # Correct format
    text = f"Title: {title_ingredients['title']} Ingredients: {title_ingredients['ingredients']}"
    
    # Tokenize and generate
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)
    
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=200,
        num_beams=5,
        early_stopping=True
    )
    
    # Decode and return
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    start = decoded.find("directions")
    return decoded[start:] if start != -1 else decoded

In [5]:
predict_recipe({"title" : "Smoothie",
                "ingredients" :"Watermelon, Kiwi, Apple and Frozen Banana Smoothie"}, tokenizer, device, model)

'directions: place all ingredients in a blender. blend until smooth. serve immediately.'